# Libraries

In [19]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
from deep_translator import GoogleTranslator

# Load Data

In [6]:
data = pd.read_csv('latestdata.csv',low_memory=False)
data

,ID,age,sex,city,province,country,latitude,longitude,geo_resolution,date_onset_symptoms,...,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
0,000-1-1,NaN,male,Shek Lei,Hong Kong,China,22.365019,114.133808,point,NaN,...,NaN,NaN,Shek Lei,NaN,NaN,Hong Kong,China,8029.0,NaN,NaN
1,000-1-10,78,male,Vo Euganeo,Veneto,Italy,45.297748,11.658382,point,NaN,...,22.02.2020,NaN,Vo' Euganeo,NaN,NaN,Veneto,Italy,8954.0,NaN,NaN
2,000-1-100,61,female,NaN,NaN,Singapore,1.353460,103.815100,admin0,NaN,...,17.02.2020,NaN,NaN,NaN,NaN,NaN,Singapore,200.0,NaN,NaN
3,000-1-1000,NaN,NaN,Zhengzhou City,Henan,China,34.629310,113.468000,admin2,NaN,...,NaN,NaN,NaN,NaN,Zhengzhou City,Henan,China,10091.0,NaN,NaN
4,000-1-10000,NaN,NaN,Pingxiang City,Jiangxi,China,27.513560,113.902900,admin2,NaN,...,NaN,NaN,NaN,NaN,Pingxiang City,Jiangxi,China,7060.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676306,010-99995,52,female,Calleria,Coronel Portillo,Peru,-8.378190,-74.539700,point,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Peru,14429.0,NaN,False
2676307,010-99996,52,female,Pueblo Libre,Lima,Peru,-12.076530,-77.067350,point,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Peru,14695.0,NaN,False
2676308,010-99997,52,female,Comas,Lima,Peru,-11.932980,-77.040850,point,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Peru,14400.0,NaN,False
2676309,010-99998,52,male,Callao,Callao,Peru,-12.000740,-77.118240,point,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Peru,14516.0,NaN,False


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676311 entries, 0 to 2676310
Data columns (total 33 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ID                        object 
 1   age                       object 
 2   sex                       object 
 3   city                      object 
 4   province                  object 
 5   country                   object 
 6   latitude                  float64
 7   longitude                 float64
 8   geo_resolution            object 
 9   date_onset_symptoms       object 
 10  date_admission_hospital   object 
 11  date_confirmation         object 
 12  symptoms                  object 
 13  lives_in_Wuhan            object 
 14  travel_history_dates      object 
 15  travel_history_location   object 
 16  reported_market_exposure  object 
 17  additional_information    object 
 18  chronic_disease_binary    bool   
 19  chronic_disease           object 
 20  source                  

NaN qiymatlar foizda

In [8]:
data.isnull().sum()/len(data)*100

ID                           0.000000
age                         78.402435
sex                         78.322512
city                        36.530919
province                    16.913729
country                      0.004297
latitude                     0.002616
longitude                    0.002616
geo_resolution               0.002616
date_onset_symptoms         90.225389
date_admission_hospital     95.657792
date_confirmation            4.053677
symptoms                    99.923327
lives_in_Wuhan              99.837911
travel_history_dates        99.902440
travel_history_location     99.655421
reported_market_exposure    99.960057
additional_information      98.286634
chronic_disease_binary       0.000000
chronic_disease             99.991967
source                      21.184533
sequence_available          99.999552
outcome                     88.514713
date_death_or_discharge     99.882375
notes_for_discussion        99.976086
location                    99.500208
admin3      

In [9]:
data.isnull().sum()

ID                                0
age                         2098293
sex                         2096154
city                         977681
province                     452664
country                         115
latitude                         70
longitude                        70
geo_resolution                   70
date_onset_symptoms         2414712
date_admission_hospital     2560100
date_confirmation            108489
symptoms                    2674259
lives_in_Wuhan              2671973
travel_history_dates        2673700
travel_history_location     2667089
reported_market_exposure    2675242
additional_information      2630456
chronic_disease_binary            0
chronic_disease             2676096
source                       566964
sequence_available          2676299
outcome                     2368929
date_death_or_discharge     2673163
notes_for_discussion        2675671
location                    2662935
admin3                      2595877
admin2                      

# Data Cleaning

### Part 1

1.Odatda judayam ko'p NaN qiymatlari bo'lgan ustunlar bilan ishlash unchalik yaxshi foyda bermaydi.Shuning uchun NaN qiymatlari miqdori 50% dan katta bo'lgan ustunlarni olib tashladim(Foiz miqdorini o'zim belgiladim)

In [10]:
data = data[[i for i in data.columns if data[i].isnull().sum()/len(data)<0.5]]

2.city,province,country,latitude,longitude va geo_resolution ustunlaridagi qiymatlar birgalikda NaN bo'lgan qatorlarni olib tashlaymiz.Chunki bu datasetda location muhim ahamiyatga ega 

In [11]:
data = data[data[['latitude','longitude','geo_resolution']].notnull().all(1)]

# Handling missing values

1.latitude va longitude dan foydalanib city,province va country ustunlaridagi qiymati NaN bo'lgan yacheykalarni to'ldirish.City va province ustunlarini to'ldirmadim bunga judayam ko'p vaqt ketarkan(lokatsiya internet bilan topiladi)

In [20]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [12]:
#Ba'zi city,province va countrylar nomi ingliz tilida yozilmagan ekan.Shuning uchun GoogleTranslator kutubxonasi bn tarjima qildim
translator = GoogleTranslator(source='auto', target='en')

In [ ]:
#city ustuni uchun
for i in data[data.city.isnull()].index:
    
    lat_i = str(data['latitude'][i])
    long_i = str(data['longitude'][i])
    
    try:
        location = geolocator.reverse(lat_i+","+long_i)
        print(location)
        address = location.raw['address']
        print(address)
        
        
        if address.get('city', ''):
            translate_city = translator.translate(address.get('city', ''))
            data.loc.__setitem__((i, ('city')), translate_city)
        else:
            pass
        
    except AttributeError:
        continue
    except TimeoutError:
        continue

In [ ]:
#province ustuni uchun
for i in data[data.province.isnull()].index:
    
    lat_i = str(data['latitude'][i])
    long_i = str(data['longitude'][i])
    
    try:
        location = geolocator.reverse(lat_i+","+long_i)
        address = location.raw['address']
        
        if address.get('province', ''):
            translate_province = translator.translate(address.get('province', ''))
            data.loc.__setitem__((i, ('province')), translate_province)
        else:
            pass
        
    except AttributeError:
        continue
    except TimeoutError:
        continue

In [10]:
#country ustuni uchun
for i in data[data.country.isnull()].index:
    
    lat_i = str(data['latitude'][i])
    long_i = str(data['longitude'][i])
    
    try:
        location = geolocator.reverse(lat_i+","+long_i)
        address = location.raw['address']
        
        if address.get('country', ''):
            translate_country = translator.translate(address.get('country', ''))
            data.loc.__setitem__((i, ('country')), translate_country)
        else:
            pass
        
    except AttributeError:
        continue
    except TimeoutError:
        continue

In [21]:
data.city.isnull().sum()

977078

In [23]:
# City ustuni NaN bo'lgan qatorlarni olib tashlaymiz
data = data[data['city'].notnull()]

In [12]:
sum(data['country']==data['country_new'])/len(data)

0.9885372804616624

In [13]:
# salkam 99% odam o'z davlatida qolganini inobatga olib bo'sh davlatlarni dastlabki qiymati bilan almashtiramiz
for i in data[data['country_new'].isnull()].index:
    data.loc.__setitem__((i, ('country_new')),data['country'][i])

In [14]:
# dastlabki va oxirgi davlatiga qarab sayohat qilgan yoki qilmaganini topamiz
for i in data[data['travel_history_binary'].isnull()].index:
    if data['country'][i] == data['country_new'][i]:
        data.loc.__setitem__((i, ('travel_history_binary')),False)
    data.loc.__setitem__((i, ('travel_history_binary')),True)

# Data Cleaning

### Part 2

In [17]:
# date_cofirmation ustunidagi nan qiymatlar bor yo'g'i 4% ni tashkil qilgani uchun boshqa ustunlardagi o'sha qatorlarni tashlab ketdim
data = data[data['date_confirmation'].notnull()]

In [21]:
# kerak bo'lmaydi deb hisoblagan ustunlarni o'chirib tashladim
# city va province ustunlaridagi nan qiymatlarni qoldirdim chunki ularning uzunlik va kengliklari bor
data = data.drop(columns=['ID','geo_resolution','source','admin_id','data_moderator_initials'])

In [24]:
data.isnull().sum()/len(data)

city                      0.000000
province                  0.001228
country                   0.000000
latitude                  0.000000
longitude                 0.000000
date_confirmation         0.000000
chronic_disease_binary    0.000000
country_new               0.000000
travel_history_binary     0.000000
dtype: float64

In [22]:
data

,city,province,country,latitude,longitude,date_confirmation,chronic_disease_binary,country_new,travel_history_binary
0,Shek Lei,Hong Kong,China,22.365019,114.133808,14.02.2020,False,China,True
1,Vo Euganeo,Veneto,Italy,45.297748,11.658382,21.02.2020,False,Italy,True
2,NaN,NaN,Singapore,1.353460,103.815100,14.02.2020,False,Singapore,True
3,Zhengzhou City,Henan,China,34.629310,113.468000,26.01.2020,False,China,True
4,Pingxiang City,Jiangxi,China,27.513560,113.902900,14.02.2020,False,China,True
...,...,...,...,...,...,...,...,...,...
2676306,Calleria,Coronel Portillo,Peru,-8.378190,-74.539700,17.05.2020,False,Peru,False
2676307,Pueblo Libre,Lima,Peru,-12.076530,-77.067350,17.05.2020,False,Peru,False
2676308,Comas,Lima,Peru,-11.932980,-77.040850,17.05.2020,False,Peru,False
2676309,Callao,Callao,Peru,-12.000740,-77.118240,17.05.2020,False,Peru,False


# Saving processed data 

In [25]:
data.to_csv('finaldata.csv',index=False)